In [1]:
import xml.dom.minidom
import xml.etree.ElementTree as ET
from xml.dom.minidom import Node
import pandas as pd

In [35]:
DOU_Periop = pd.read_csv("DOU CY2020 PeriOp Volume.csv")
Q1 = DOU_Periop[DOU_Periop.Month.isin([1,2,3])]
Q2 = DOU_Periop[DOU_Periop.Month.isin([4,5,6])]
Q3 = DOU_Periop[DOU_Periop.Month.isin([7,8,9])]
Q4 = DOU_Periop[DOU_Periop.Month.isin([10,11,12])]

In [2]:
PeriOp_Volume = pd.read_csv("2021_Q2 PeriOp Volume.csv")

In [3]:
# Fill NaN with 0 and change data type from float to integer
PeriOp_Volume.TotalNumberofPreoperativePatients = PeriOp_Volume.TotalNumberofPreoperativePatients.fillna(0.0).astype(int)
PeriOp_Volume.TotalNumberofPostoperativePatients = PeriOp_Volume.TotalNumberofPostoperativePatients.fillna(0.0).astype(int)
PeriOp_Volume.PostoperativePatientLengthofStayinHours = PeriOp_Volume.PostoperativePatientLengthofStayinHours.fillna(0.0).astype(int)
PeriOp_Volume.TotalSurgicalCases = PeriOp_Volume.TotalSurgicalCases.fillna(0.0).astype(int)
PeriOp_Volume.TotalSurgicalMinutes = PeriOp_Volume.TotalSurgicalMinutes.fillna(0.0).astype(int)
PeriOp_Volume.TotalNumberofPatients = PeriOp_Volume.TotalNumberofPatients.fillna(0.0).astype(int)

## Periop Pt Volume Function

In [4]:
# The variables needed for this function are the: Hospital abbreviation, Hospital NDNQI template, Hospital NDNQI
# code, Calendar year being submitted, and Calendar quarter being submitted
def NDNQI_Periop_Pt_Volume_XML(Selected_Hosp,Hosp_XML_Template,Hosp_NDNQI_Code,Calendar_Year,Calendar_Quarter):
    # This section is to update the Peri-Operative Patient Volumes
 
    # The first step takes only the data for this selected hospital
    Hosp_data = PeriOp_Volume[PeriOp_Volume.Hospital == Selected_Hosp]
    # A List of this Hospital's Unit Names
    Hosp_Units = list(set(Hosp_data.UnitName))

    #This is the current XML template for this hospital and their NDNQI units for this metric
    tree = xml.dom.minidom.parse(Hosp_XML_Template)
    # This is each unit node of the xml file
    unit_nodes = tree.getElementsByTagName("UnitYrQtr")

   # Make sure the correct NDNQI Hospital code, year, and quarter are listed
    H = tree.getElementsByTagName("HospitalCode")[0].childNodes[0].nodeValue = Hosp_NDNQI_Code
    Y = tree.getElementsByTagName("Year")[0].childNodes[0].nodeValue = Calendar_Year
    Q = tree.getElementsByTagName("Quarter")[0].childNodes[0].nodeValue = Calendar_Quarter


    # This loops through each separate UnitName from the Template XML file and replaces it the correct NDNQI
    # name for the units. 
    for idx, unit in enumerate(unit_nodes):
        unit.getElementsByTagName('UnitName')[0].childNodes[0].nodeValue = Hosp_Units[idx]

        # Now grab each of the 3 months for that unit from the Hospital data
        new_data = Hosp_data[Hosp_data.UnitName == Hosp_Units[idx]]
        # Sort the data by month
        new_data = new_data.sort_values('Month')

        # Take each xml Month and replace it with each listed month from the new_data for that unit
        month_nodes = unit.getElementsByTagName("UnitMonth")
        for idx2, month in enumerate(month_nodes):
            month.getElementsByTagName('Month')[0].childNodes[0].nodeValue = new_data.Month.iloc[idx2]

            for column in new_data.columns[3:]:

                new_value = new_data[column].iloc[idx2]
                # If the data is a 0 then it should be a null in the xml file
                if new_value == 0:
                    # I first remove the prior text from that element
                    element = month.getElementsByTagName(column)[0]
                    element_text = element.childNodes[0]
                    element.removeChild(element_text)
                    # Then I set that element to a null attribute
                    element.setAttribute('xsi:nil', 'true')
                else:
                    # If the data is not a 0 then replace that metric with the new value.
                    month.getElementsByTagName(column)[0].childNodes[0].nodeValue = new_value
                    

    # Now, we need to write and save this XML file as its own NDNQI Quarter and Year and Hospital to upload
    # into NDNQI.
    tree.writexml(open('Q{}_{} {} PeriOp Pt Volume.xml'.format(Calendar_Quarter, Calendar_Year,Selected_Hosp), 'w'))

## EJCH

In [5]:
NDNQI_Periop_Pt_Volume_XML('EJCH','2020 EJCH PeriOp Pt Volume Template.xml','YTD069',2021,2)

In [6]:
EJCH_data = PeriOp_Volume[PeriOp_Volume.Hospital == 'EJCH']
EJCH_data

,Hospital,UnitName,Month,TotalNumberofPostoperativePatients,TotalNumberofPatients,TotalSurgicalCases,PostoperativePatientLengthofStayinHours,TotalSurgicalMinutes,TotalNumberofPreoperativePatients
13,EJCH,JC PACU,5,441,0,0,647,0,0
14,EJCH,JC OR,6,0,0,497,0,66084,0
16,EJCH,JC PACU,6,510,0,0,754,0,0
19,EJCH,JC OR,5,0,0,435,0,56899,0
21,EJCH,JC PACU,4,451,0,0,639,0,0
23,EJCH,JC SDS,6,0,549,0,0,0,0
26,EJCH,JC SDS,4,0,535,0,0,0,0
32,EJCH,JC OR,4,0,0,441,0,55672,0
38,EJCH,JC SDS,5,0,544,0,0,0,0


## EJSH

In [7]:
NDNQI_Periop_Pt_Volume_XML('ESJH','2020 ESJH PeriOp Pt Volume Template.xml','XAR317',2021,2)

In [8]:
ESJH_data = PeriOp_Volume[PeriOp_Volume.Hospital == 'ESJH']
ESJH_data

,Hospital,UnitName,Month,TotalNumberofPostoperativePatients,TotalNumberofPatients,TotalSurgicalCases,PostoperativePatientLengthofStayinHours,TotalSurgicalMinutes,TotalNumberofPreoperativePatients
9,ESJH,ESJH Operating Room,4,0,0,1127,0,181902,0
20,ESJH,"ESJH Pre-Postoperative (PAT, Pre-Op, PSCU, Gro...",6,1192,0,0,1889,0,1191
22,ESJH,ESJH Operating Room,6,0,0,1204,0,192100,0
27,ESJH,"ESJH Pre-Postoperative (PAT, Pre-Op, PSCU, Gro...",5,1092,0,0,1940,0,1140
35,ESJH,"ESJH Pre-Postoperative (PAT, Pre-Op, PSCU, Gro...",4,1126,0,0,1868,0,1118
39,ESJH,ESJH Operating Room,5,0,0,1137,0,184684,0


## EUOSH

In [9]:
NDNQI_Periop_Pt_Volume_XML('EUOSH','2020 EUOSH PeriOp Pt Volume Template.xml','SBI556',2021,2)

In [10]:
EUOSH_data = PeriOp_Volume[PeriOp_Volume.Hospital == 'EUOSH']
EUOSH_data

,Hospital,UnitName,Month,TotalNumberofPostoperativePatients,TotalNumberofPatients,TotalSurgicalCases,PostoperativePatientLengthofStayinHours,TotalSurgicalMinutes,TotalNumberofPreoperativePatients
0,EUOSH,OSH POHA/PAT,4,0,0,0,0,0,362
2,EUOSH,OSH POHA/PAT,6,0,0,0,0,0,391
5,EUOSH,OSH POHA/PAT,5,0,0,0,0,0,332
7,EUOSH,OSH PACU,6,440,0,0,973,0,0
8,EUOSH,OSH OR,5,0,0,371,0,67826,0
10,EUOSH,OSH OR,4,0,0,391,0,71215,0
28,EUOSH,OSH PACU,5,365,0,0,750,0,0
29,EUOSH,OSH OR,6,0,0,438,0,76588,0
36,EUOSH,OSH PACU,4,377,0,0,746,0,0


## EUH

In [11]:
NDNQI_Periop_Pt_Volume_XML('EUH','2020 EUH PeriOp Pt Volume Template.xml','JXE432',2021,2)

In [12]:
EUH_data = PeriOp_Volume[PeriOp_Volume.Hospital == 'EUH']
EUH_data

,Hospital,UnitName,Month,TotalNumberofPostoperativePatients,TotalNumberofPatients,TotalSurgicalCases,PostoperativePatientLengthofStayinHours,TotalSurgicalMinutes,TotalNumberofPreoperativePatients
4,EUH,EUH PACU,6,1456,0,0,1862,0,0
6,EUH,EUH POHA,4,0,0,0,0,0,2208
11,EUH,EUH OR,5,0,0,1088,0,223970,0
12,EUH,EUH PACU,5,1338,0,0,1802,0,0
18,EUH,EUH PACU,4,1363,0,0,1827,0,0
30,EUH,EUH POHA,5,0,0,0,0,0,2236
33,EUH,EUH OR,4,0,0,1053,0,220052,0
37,EUH,EUH OR,6,0,0,1129,0,228185,0
40,EUH,EUH POHA,6,0,0,0,0,0,2285


## EUHM

In [13]:
NDNQI_Periop_Pt_Volume_XML('EUHM','2020 EUHM PeriOp Pt Volume Template.xml','PVF155',2021,2)

In [14]:
EUHM_data = PeriOp_Volume[PeriOp_Volume.Hospital == 'EUHM']
EUHM_data

,Hospital,UnitName,Month,TotalNumberofPostoperativePatients,TotalNumberofPatients,TotalSurgicalCases,PostoperativePatientLengthofStayinHours,TotalSurgicalMinutes,TotalNumberofPreoperativePatients
1,EUHM,EUHM Pre-op PAT,5,0,0,0,0,0,1131
3,EUHM,EUHM PACU,6,1595,0,0,1817,0,0
15,EUHM,EUHM Pre-op PAT,4,0,0,0,0,0,1214
17,EUHM,EUHM PACU,5,1530,0,0,1834,0,0
24,EUHM,EUHM OR,4,0,0,1284,0,218621,0
25,EUHM,EUHM Pre-op PAT,6,0,0,0,0,0,1225
31,EUHM,EUHM PACU,4,1629,0,0,1983,0,0
34,EUHM,EUHM OR,5,0,0,1220,0,210602,0
41,EUHM,EUHM OR,6,0,0,1297,0,235773,0


## EDH

In [15]:
NDNQI_Periop_Pt_Volume_XML('EDH','2021 EDH PeriOp Pt Volume Template.xml','NMW346',2021,2)

In [16]:
EDH_data = PeriOp_Volume[PeriOp_Volume.Hospital == 'EDH']
EDH_data

,Hospital,UnitName,Month,TotalNumberofPostoperativePatients,TotalNumberofPatients,TotalSurgicalCases,PostoperativePatientLengthofStayinHours,TotalSurgicalMinutes,TotalNumberofPreoperativePatients
43,EDH,EDH Operating Room,4,0,0,833,0,109300,0
44,EDH,EDH POHA,4,0,0,0,0,0,685
45,EDH,EDH Operating Room,5,0,0,839,0,102700,0
46,EDH,EDH Operating Room,6,0,0,868,0,113800,0
47,EDH,EDH POHA,6,0,0,0,0,0,729
48,EDH,EDH PAT,4,0,0,0,0,0,728
49,EDH,EDH PAT,5,0,0,0,0,0,712
50,EDH,EDH PACU,4,679,0,0,1023,0,0
54,EDH,EDH PACU,6,688,0,0,1161,0,0
56,EDH,EDH PACU,5,646,0,0,105,0,0


## EHH

In [17]:
NDNQI_Periop_Pt_Volume_XML('EHH','2021 EHH PeriOp Pt Volume Template.xml','WXX116',2021,2)

In [18]:
EHH_data = PeriOp_Volume[PeriOp_Volume.Hospital == 'EHH']
EHH_data

,Hospital,UnitName,Month,TotalNumberofPostoperativePatients,TotalNumberofPatients,TotalSurgicalCases,PostoperativePatientLengthofStayinHours,TotalSurgicalMinutes,TotalNumberofPreoperativePatients
42,EHH,EHH Special Procedures,6,0,0,90,0,3602,0
51,EHH,EHH POHA,6,0,198,0,0,0,0
52,EHH,EHH Operating Room,4,0,0,125,0,12000,0
53,EHH,EHH Special Procedures,5,0,0,83,0,2996,0
55,EHH,EHH Operating Room,6,0,0,134,0,12300,0
57,EHH,EHH Special Procedures,4,0,0,73,0,2412,0
58,EHH,EHH POHA,5,0,193,0,0,0,0
59,EHH,EHH POHA,4,0,201,0,0,0,0
61,EHH,EHH Operating Room,5,0,0,125,0,11700,0
